<a href="https://colab.research.google.com/github/damilojohn/Playlist_Generator/blob/main/Copy_of_Generating_Afrobeats_embeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#installing necessary libraries
!pip install sentence-transformers
!pip install wandb

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 21.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 37.0 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=4aa730067ee2dc5d4d66eb6227a76ab713082af2ae93640128227e9a4b37293a
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
#songs csv 
!wget https://raw.githubusercontent.com/damilojohn/Playlist_Generator/main/spotify_transformed.csv

--2023-02-05 07:06:00--  https://raw.githubusercontent.com/damilojohn/Playlist_Generator/main/spotify_transformed.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 167653 (164K) [text/plain]
Saving to: ‘spotify_transformed.csv’

spotify_transformed 100%[===================>] 163.72K  --.-KB/s    in 0.02s   

2023-02-05 07:06:00 (6.93 MB/s) - ‘spotify_transformed.csv’ saved [167653/167653]



In [ ]:
#setting up wandb
!wandb login
import wandb

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
config = {
    'Name' : 'Embeddings for afrobeats lyrics',
    'Dataset' : 'spotify api lyrics',
    'Model' : 'msmarco-miniLM-pretrained',
    'notes':'No text cleaning was performed yet.creating a baseline performance'
}
wandb.init(project='Playlist Generator',entity='damilojohn',config=config)

In [3]:
from sentence_transformers import SentenceTransformer,util
from tqdm.auto import tqdm
import pandas as pd 
import transformers
import pickle 
from huggingface_hub import hf_hub_download
import re

In [4]:
#loading song lyrics into a dataset object 
data_path = '/content/spotify_transformed.csv'

In [5]:
embedder = SentenceTransformer('msmarco-distilbert-base-v4')

In [6]:
songs= pd.read_csv(data_path)

In [9]:
sample=songs

In [10]:
def clean_verses(lyrics_uncleaned):
  ''' takes a list of verses as input and removes useless characters such as /n and stuff '''
  lyrics_cleaned = lyrics_uncleaned.split('\n')
  lyrics_cleaned = [i for i in lyrics_cleaned if i]
  for i,lyrics in enumerate(lyrics_cleaned) :
    if not lyrics[0].isalpha():
      del lyrics_cleaned[i]
  return lyrics_cleaned 


In [11]:
sample['for_lyrics'] = sample['for_lyrics'].apply(clean_verses)

In [12]:
sample = sample[(sample['for_lyrics'].str.len() != 0 )]

In [ ]:
verses = []
for i in sample['for_lyrics'].values:
  verses.append(i)
verses = [i for i in verses if i] 
new_verses = [] 
for i in verses :
  for j in i :
    new_verses.append(j)
new_verses

In [34]:
new_verses = {i:k for i,k in enumerate(new_verses)}

In [35]:
verse_df = pd.DataFrame.from_dict(new_verses,orient='index')

In [36]:
verse_df.reset_index(level=0,inplace=True)

In [38]:
verse_df.to_csv('/content/verses.csv')

In [24]:
new_dict

{'Got a bag full of feelings': '2 Sugar (feat. Ayra Starr)',
 'And a house full of regrets': '2 Sugar (feat. Ayra Starr)',
 "This baggages can't fit in no plane, don't mean to digress, hmm": '2 Sugar (feat. Ayra Starr)',
 "I wanna know what's the reason why you moving slow?": '2 Sugar (feat. Ayra Starr)',
 'As you move no dey cross my lane': '2 Sugar (feat. Ayra Starr)',
 'Shori nonsense, ma gbe sunmomi': '2 Sugar (feat. Ayra Starr)',
 'Ma gbe sunmomi': '2 Sugar (feat. Ayra Starr)',
 'If you getti problem, ma gbe sunmomi': '2 Sugar (feat. Ayra Starr)',
 "I dey fight my own demons, don't you see?": '2 Sugar (feat. Ayra Starr)',
 "Don't know, don't you see?": '2 Sugar (feat. Ayra Starr)',
 'Oya mami jo, jogo, jogodo, mami go': '2 Sugar (feat. Ayra Starr)',
 'And I no fit lie for you, mami, oh, yo': '2 Sugar (feat. Ayra Starr)',
 'Go low, ko pami oh': '2 Sugar (feat. Ayra Starr)',
 'And me no dey love when dem try me, oh': '2 Sugar (feat. Ayra Starr)',
 'Say my kind life no dey funny, oh'

In [14]:
dictt=dict(zip(sample.song_name,sample.for_lyrics))

In [15]:
new_dict={}
for i,j in dictt.items():
    for k in j:
        new_dict[k]=i

In [16]:
embeddings = embedder.encode(new_verses,)
assert len(new_verses) == len(embeddings) , 'embeddings and lyrics are mismatched '

In [30]:
#trying out semantic search 
query = ['relax and have fun']
query_embedding = embedder.encode(query)
hits = util.semantic_search(query_embedding,embeddings,top_k = 6)
hits

[[{'corpus_id': 4144, 'score': 0.660988986492157},
  {'corpus_id': 4140, 'score': 0.5234023928642273},
  {'corpus_id': 1997, 'score': 0.47801321744918823},
  {'corpus_id': 2015, 'score': 0.47801321744918823},
  {'corpus_id': 2007, 'score': 0.47801321744918823},
  {'corpus_id': 1974, 'score': 0.47801321744918823}]]

In [31]:
for hit in hits[0]:
  print(new_dict[new_verses[hit['corpus_id']]])

Jericho
Jericho
Likkle Riddim
Likkle Riddim
Likkle Riddim
Likkle Riddim


In [ ]:
with open(path='/content/verses.pkl','wb') as pk:
  pickle.dump(new_verses,pk)

In [51]:
mappings = pd.DataFrame.from_dict(new_dict,orient='index')

In [57]:
mappings.to_csv('/content/song_mappings.csv')

In [47]:
import pickle
path = '/content/em.pkl'
with open(path,'wb') as pk:
  pickle.dump(embeddings,pk)

In [48]:
from huggingface_hub import login 
login()

Token is valid.
Your token has been saved in your configured git credential helpers (store).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [49]:
from huggingface_hub import hf_hub_download